# WHY FAKE NEWS IS A PROBLEM?
**Fake news refers to misinformation, disinformation or mal-information which is spread through word of mouth and traditional media and more recently through digital forms of communication such as edited videos, memes, unverified advertisements and social media propagated rumours.Fake news spread through social media has become a serious problem, with the potential of it resulting in mob violence, suicides etc as a result of misinformation circulated on social media.**
![image.png](attachment:image.png)

# BRIEF DESCRIPTION OF DATASET
**This dataset consists of about 40000 articles consisting of fake as well as real news. Our aim is train our model so that it can correctly predict whether a given piece of news is real or fake.The fake and real news data is given in two separate datasets with each dataset consisting around 20000 articles each.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls
%cd /content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset

drive  sample_data
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


In [4]:
%cd fake-and-real-news-dataset/

[Errno 2] No such file or directory: 'fake-and-real-news-dataset/'
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


# LOADING THE NECESSARY LIBRARIES

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

# IMPORTING THE DATASET

In [6]:
# true = pd.read_csv("../input/fake-and-real-news-dataset/True.csv")
# false = pd.read_csv("../input/fake-and-real-news-dataset/Fake.csv")
true = pd.read_csv("True.csv")
false = pd.read_csv("Fake.csv")

# DATA VISUALIZATION AND PREPROCESSING

**Import data in tf dataset**

In [7]:
true_dataset = tf.data.experimental.CsvDataset(filenames = ['True.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)
false_dataset = tf.data.experimental.CsvDataset(filenames = ['Fake.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)

**Join text and title**

In [8]:
def preprocess_join_sentence(s1, s2):
    ret = tf.strings.reduce_join([s1, s2],separator=" ", axis=-1)
    return ret

true_dataset = true_dataset.map(preprocess_join_sentence)
false_dataset = false_dataset.map(preprocess_join_sentence)

### Add labels to dataset

In [11]:
cat_true = tf.data.Dataset.from_tensor_slices(np.ones(len(list(true_dataset)), dtype=int))
cat_false = tf.data.Dataset.from_tensor_slices(np.zeros(len(list(false_dataset)), dtype=int))

In [12]:
true_dataset = tf.data.Dataset.zip((true_dataset, cat_true))
false_dataset = tf.data.Dataset.zip((false_dataset, cat_false))

In [93]:
full_dataset = false_dataset.concatenate(true_dataset)

**Formate sentences**

In [98]:
# def preprocess_sentence(sentence, cat):
def preprocess_sentence(sentence, label):
  ret = tf.strings.lower(sentence)
  # ret = BeautifulSoup(ret, "html.parser").get_text()
  ret = tf.strings.strip(ret)
  ret = tf.strings.regex_replace(ret, '<[^<]+?>', '') # Removing HTML chars
  ret = tf.strings.regex_replace(ret, '\[[^]]*\]', '') # Removing the square brackets
  ret = tf.strings.regex_replace(ret, 'http\S+', '') # Removing URL's
  return ret, label

full_dataset = full_dataset.map(preprocess_sentence)

In [99]:
for idx, row in enumerate(full_dataset):
    print (row)
    if idx > 3:
        break

(<tf.Tensor: shape=(), dtype=string, numpy=b'donald trump sends out embarrassing new year\xe2\x80\x99s eve message; this is disturbing donald trump just couldn t wish all americans a happy new year and leave it at that. instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  the former reality show star had just one job to do and he couldn t do it. as our country rapidly grows stronger and smarter, i want to wish all of my friends, supporters, enemies, haters, and even the very dishonest fake news media, a happy and healthy new year,  president angry pants tweeted.  2018 will be a great year for america! as our country rapidly grows stronger and smarter, i want to wish all of my friends, supporters, enemies, haters, and even the very dishonest fake news media, a happy and healthy new year. 2018 will be a great year for america!  donald j. trump (@realdonaldtrump) december 31, 2017trump s tweet went down about as welll as you d expect.what ki

In [ ]:
!pip install -q tensorflow-text


### Glove

In [15]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Build vocabulary and relevent vector
words = []
# Adding 2 empty vector of dimension 100 which will be used in padding and unknown token(word)
# According to previous knowledge i have idea that it contains 400,000 words
# Added extra 2 rows because when we will use inside TextVectorization add blank i.e. '' and ['UNK'] inside vocabulary
vectors = np.zeros((400002, 50))

# with open(f"../glove-twitter/glove.twitter.27B.100d.txt") as f:
with open(f"../glove-twitter/glove.6B.50d.txt") as f:
    lines = f.readlines()
    for idx, line in enumerate(lines):
        split_line = line.split()
        words.append(split_line[0])
        vectors[idx+2] = split_line[1:]

print(f"Number of words are {len(words)}")
print(f"Shape of vector is {vectors.shape}")

Number of words are 400000
Shape of vector is (400002, 50)


In [18]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# TextVectorization hyper parameter
max_vocab = 400002
max_len = 300
vectorize_layer_glove = TextVectorization(max_tokens=max_vocab,
                                                    standardize="lower_and_strip_punctuation",
                                                    split="whitespace",
                                                    output_mode="int",
                                                    output_sequence_length=max_len)


# Vocabulary set into layer
vectorize_layer_glove.set_vocabulary(words)
print(f"Top 10 words {vectorize_layer_glove.get_vocabulary()[:10]}")
print(f"Some words {vectorize_layer_glove.get_vocabulary()[-10010:-10000]} \n")

Top 10 words ['', '[UNK]', 'the', ',', '.', 'of', 'to', 'and', 'in', 'a']
Some words ['czermin', 'basketbal', 'marcheschi', '10-cents', 'stafon', 'haripal', 'rahmane', 'valdivielso', 'schleifstein', 'pyongyong'] 



In [19]:
# vectorize_layer_glove.adapt(full_dataset)
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  # return vectorize_layer_glove(text)
  return vectorize_layer_glove(text), label

train_data_tokenized = full_dataset.map(int_vectorize_text)

In [ ]:
for idx, row in enumerate(train_data_tokenized):
    print (row)
    if idx > 3:
        break


In [23]:
len(vectors[0])

50

### Prepare data to train

In [44]:
# Model params
batch_size = 256
epochs = 10
embed_size = 50
maxlen = 300
max_features = 400002



**Model**

In [87]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
# model_glove.add(tf.keras.layers.Embedding(max_vocab, 50, weights=[vectors], trainable=False))
model.add(Embedding(max_features, output_dim=embed_size, weights=[vectors], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)


In [88]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 300, 50)           20000100  
_________________________________________________________________
lstm_10 (LSTM)               (None, 300, 128)          91648     
_________________________________________________________________
lstm_11 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 20,143,269
Trainable params: 143,169
Non-trainable params: 20,000,100
_________________________________________________________________


### TODO : add validation dataset and shuffle data

In [ ]:
# dataset = train_data_tokenized.batch(batch_size).repeat()

In [89]:
train_data = train_data_tokenized.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

In [ ]:
for idx, row in enumerate(train_data):
    print (row)
    if idx > 3:
        break

In [90]:
# Trains for 5 epochs
model.fit(train_data, epochs=5, steps_per_epoch=100)

Epoch 1/5
100/100 [==============================] - 81s 762ms/step - loss: 0.0626 - accuracy: 0.9485
Epoch 2/5
100/100 [==============================] - 79s 787ms/step - loss: 6.9772e-08 - accuracy: 1.0000
Epoch 3/5
100/100 [==============================] - 77s 766ms/step - loss: 6.9235e-08 - accuracy: 1.0000
Epoch 4/5
100/100 [==============================] - 77s 766ms/step - loss: 6.8215e-08 - accuracy: 1.0000
Epoch 5/5
100/100 [==============================] - 76s 764ms/step - loss: 6.7374e-08 - accuracy: 1.0000


In [ ]:
# old code -> to copy
# history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

# OLD notebook

# OLD notebook

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow_text as text



def remove_stopwords(sentence):
  final_text = []
  tokenizer = text.WhitespaceTokenizer()
  tokens = tokenizer.tokenize([sentence.encode('UTF-8')])
  unigram = text.ngrams(tokens, 1, reduction_type=text.Reduction.STRING_JOIN)

  for i in unigram:
    print(i)
    if i not in stop:
        final_text.append(unigram)
  return " ".join(final_text)
  # return preprocess_sentence(sentence1), preprocess_sentence(sentence2)

def drop_stop_words (text):
    stop = set(stopwords.words('english'))
    nltk.download('punkt')
    punctuation = list(string.punctuation)
    stop.update(punctuation)
    words = word_tokenize(text)
    a = ''
    for index, txt in enumerate(words):
        # a = ' '.join([w for w in words if w not in stop_words ] )
        a = tf.strings.reduce_join([w for w in words if w not in stop_words ],separator=" ", axis=-1)
    return a


In [ ]:
sentence = "She is a very famous girl named Jane"
# remove_stopwords(sentence, stop)
my_preprocessed_dataset = joined_dataset.map(remove_stopwords)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
from nltk.corpus import stopwords
import tensorflow_text as text

tokenizer = text.WhitespaceTokenizer()
tokenized_docs = joined_dataset.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator))
print(next(iterator))

stop_words = set(stopwords.words('english'))
for index, txt in enumerate(tokenized_docs):
  print(txt)
  # a = tf.strings.reduce_join([w for w in tokenized_docs if w not in stop_words ],separator=" ", axis=-1)



In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['text']=df['text'].apply(denoise_text)

In [ ]:
my_preprocessed_dataset = true_dataset.map(preprocess)

In [ ]:
for idx, row in enumerate(my_preprocessed_dataset):
    print (row[0])
    if idx > 3:
        break

In [ ]:
def preprocess_join(sentence1, sentence2):
  def preprocess_join_sentence(s1, s2):
    ret = tf.strings.reduce_join([s1, s2],separator=" ", axis=-1)
    return ret
  return preprocess_join_sentence(sentence1, sentence2)


In [ ]:
joined_dataset = true_dataset.map(preprocess_join)

In [ ]:
for idx, row in enumerate(joined_dataset):
    print (row)
    if idx > 3:
        break

In [ ]:
joined_dataset

In [ ]:
true.head()

In [ ]:
false.head()

In [ ]:
true['category'] = 1
false['category'] = 0

In [ ]:
df = pd.concat([true,false]) #Merging the 2 datasets
df.head()

In [ ]:
sns.set_style("darkgrid")
sns.countplot(df.category)

**SO, WE CAN SEE THAT THE DATASET IS BALANCED**

In [ ]:
df.head()

In [ ]:
pd.set_option('display.max_colwidth', -1)
df[-3:]

In [ ]:
df.isna().sum() # Checking for nan Values

In [ ]:
df.title.count()

In [ ]:
df.subject.value_counts()

**MERGING ALL THE TEXT DATA INTO 1 COLUMN i.e. 'text'**

In [ ]:
plt.figure(figsize = (12,8))
sns.set(style = "whitegrid",font_scale = 1.2)
chart = sns.countplot(x = "subject", hue = "category" , data = df)
chart.set_xticklabels(chart.get_xticklabels(),rotation=90)

**SINCE THE TOPICS IN SUBJECT COLUMN ARE DIFFERENT FOR BOTH CATEGORIES, HENCE WE HAVE TO EXCLUDE IT FROM FINAL TEXT COLUMN**

In [ ]:
df['text'] = df['text'] + " " + df['title']
del df['title']
del df['subject']
del df['date']

**WHAT ARE STOPWORDS?**

**Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. For example, the words like the, he, have etc. Such words are already captured this in corpus named corpus. We first download it to our python environment.**

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

**DATA CLEANING**

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['text']=df['text'].apply(denoise_text)

In [ ]:
text_samples = df[:20]
text_samples

In [ ]:
df['text'][0]

In [ ]:
df.text[0]

## Split dataset

**Splitting the data into 2 parts - training and testing data**

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.text,df.category,random_state = 0)

In [ ]:
samples = x_train[:20]
samples

In [ ]:
y_train[:20]

In [ ]:
max_features = 10000
maxlen = 300

**Tokenizing Text -> Repsesenting each word by a number**

**Mapping of orginal word to number is preserved in word_index property of tokenizer**

**Tokenized applies basic processing like changing it to lower case, explicitely setting that as False**

**Lets keep all news to 300, add padding to news with less than 300 words and truncating long ones**

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
x_train

In [ ]:
x_train[0]

In [ ]:
tokenizer

### Import tokenizer

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%cd ../

In [ ]:
import pickle 

# loading
with open('tokenizer.pickle', 'rb') as handle:
    t = pickle.load(handle)

In [ ]:
tokenized_samples = t.texts_to_sequences(samples)
pad_tok_samples = sequence.pad_sequences(tokenized_samples, maxlen=maxlen)

In [ ]:
pad_tok_samples

In [ ]:
tokenized_samples = tokenizer.texts_to_sequences(samples)
pad_tok_samples = sequence.pad_sequences(tokenized_samples, maxlen=maxlen)

In [ ]:
s = sequence.pad_sequences(tokenized_samples, 300)

In [ ]:
s

In [ ]:
pad_tok_samples

In [ ]:
df = pd.DataFrame(x_train)
df

In [ ]:
!ls

In [ ]:
df.values.tolist()[0]

In [ ]:
df.to_csv('train_data.csv')

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

# Introduction to GloVe
**GloVe method is built on an important idea,
You can derive semantic relationships between words from the co-occurrence matrix.
Given a corpus having V words, the co-occurrence matrix X will be a V x V matrix, where the i th row and j th column of X, X_ij denotes how many times word i has co-occurred with word j. An example co-occurrence matrix might look as follows.**
![image.png](attachment:image.png)

**The co-occurrence matrix for the sentence “the cat sat on the mat” with a window size of 1. As you probably noticed it is a symmetric matrix.
How do we get a metric that measures semantic similarity between words from this? For that, you will need three words at a time. Let me concretely lay down this statement.**

**![image.png](attachment:image.png)
The behavior of P_ik/P_jk for various words
Consider the entity
P_ik/P_jk where P_ik = X_ik/X_i
Here P_ik denotes the probability of seeing word i and k together, which is computed by dividing the number of times i and k appeared together (X_ik) by the total number of times word i appeared in the corpus (X_i).
You can see that given two words, i.e. ice and steam, if the third word k (also called the “probe word”),
is very similar to ice but irrelevant to steam (e.g. k=solid), P_ik/P_jk will be very high (>1),
is very similar to steam but irrelevant to ice (e.g. k=gas), P_ik/P_jk will be very small (<1),
is related or unrelated to either words, then P_ik/P_jk will be close to 1
So, if we can find a way to incorporate P_ik/P_jk to computing word vectors we will be achieving the goal of using global statistics when learning word vectors.**

**Source Credits - https://towardsdatascience.com/light-on-math-ml-intuitive-guide-to-understanding-glove-embeddings-b13b4f19c010**

In [ ]:
% cd ../

In [ ]:
EMBEDDING_FILE = 'glove-twitter/glove.twitter.27B.100d.txt'


In [ ]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
#change below line if computing normal stats is too slow
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

**Some Model Parameters**

In [ ]:
batch_size = 256
epochs = 10
embed_size = 100

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)

# TRAINING THE MODEL

In [ ]:
x_train.astype

### GPU verification


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = epochs , callbacks = [learning_rate_reduction])

### Save the model

In [ ]:
model.save('model.h5')

### Load the model

In [ ]:
model = keras.models.load_model('model.h5')

### Predict samples

In [ ]:
samples.index

In [ ]:
model.predict_classes(pad_tok_samples)

In [ ]:
y_train[:20]

# ANALYSIS AFTER TRAINING OF MODEL

In [ ]:
X_test

In [ ]:
for i in range(10):
  print(model.predict(X_test[i]))

In [ ]:
len(model.predict_classes([X_test]))

In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(x_train[:10],y_train[:10])[1]*100 , "%")
print("Accuracy of the model on Testing Data is - " , model.evaluate(X_test[:10],y_test[:10])[1]*100 , "%")

In [ ]:
y_train[:5]

In [ ]:
X_test[3].reshape(1,300)

In [ ]:
predictions = model.predict_classes(X_test[:10].reshape(-1,300))
print("predictions shape:", predictions.shape)


In [ ]:
predictions

In [ ]:
epochs = [i for i in range(10)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Testing Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'go-' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'ro-' , label = 'Testing Loss')
ax[1].set_title('Training & Testing Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

In [ ]:
pred = model.predict_classes(X_test)
pred[:5]

In [ ]:
print(classification_report(y_test, pred, target_names = ['Fake','Not Fake']))

In [ ]:
cm = confusion_matrix(y_test,pred)
cm

In [ ]:
cm = pd.DataFrame(cm , index = ['Fake','Original'] , columns = ['Fake','Original'])

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='' , xticklabels = ['Fake','Original'] , yticklabels = ['Fake','Original'])
plt.xlabel("Predicted")
plt.ylabel("Actual")

**PLS UPVOTE THIS NOTEBOOK IF YOU LIKE IT! THANKS FOR YOUR TIME !**